In [1]:
# EOMs
%reset
#import libraries
import sympy as sym
import numpy as np
import pickle as pickle

sym.init_printing()
from IPython.display import display #for pretty printing

# Symbols --------------------------------------------------
bases = ['body']

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
#Parameters
masses = {}

g = sym.symbols('g')

for base in bases:
    masses.update({(base): sym.symbols('m_{0}'.format(base))})

#gen coordinates
x_body,y_body = sym.symbols(['x_body','y_body'])
dx_body,dy_body = sym.symbols(['\dot{\\x}_{body}','\dot{\\y}_{body}'])
ddx_body,ddy_body = sym.symbols(['\ddot{\\x}_{body}','\ddot{\\y}_{body}'])


In [3]:
q = sym.Matrix([x_body]+[y_body])
dq = sym.Matrix([dx_body]+[dy_body])
ddq = sym.Matrix([ddx_body]+[ddy_body])

In [4]:
q

⎡x_body⎤
⎢      ⎥
⎣y_body⎦

In [5]:
dq

⎡\dot{\x}_{body}⎤
⎢               ⎥
⎣\dot{\y}_{body}⎦

In [6]:
ddq

⎡\ddot{\x}_{body}⎤
⎢                ⎥
⎣\ddot{\y}_{body}⎦

In [7]:
''' Setup positions (to COM) and velocities'''

r_bases = {}
dr_bases = {}

# position to COM of each link - where the mass is approximated to be
r_bases.update({'body': sym.Matrix([[x_body,
                                     y_body]]).transpose()})

dr_bases.update({'body': r_bases['body'].jacobian(q)*dq})

In [8]:
T_bases = {}
V_bases = {}

# Kinetic Energy
T_bases.update({'body': 0.5*sym.Matrix([[masses['body'],masses['body']]])*sym.matrix_multiply_elementwise(dr_bases['body'],dr_bases['body']) })

# Potential Energy
for base in bases:
    V_bases.update({base: masses[base]*g*r_bases[base][1]})

Ttot = T_bases['body']
Vtot = V_bases['body']
        
Ttot = Ttot[0]

In [9]:
# Manipulator Equation (M*ddq + C + G = Tau + Q)

# M Matrix
M = sym.zeros(len(q),len(q))

for i in range(len(q)):
    for j in range(len(q)):
        M[i,j] = sym.diff(sym.diff(Ttot,dq[i]),dq[j])

M = sym.simplify(M)

M_size = int(np.sqrt(len(M))) # assumes square matrix (len(M) = 4x4 = 16)

dM = sym.zeros(M_size,M_size)

for i in range(M_size):
    for j in range(M_size):
        dM[i,j] = sym.Matrix([M[i,j]]).jacobian(q)*dq

# C Matrix
C = dM*dq - (sym.Matrix([Ttot]).jacobian(q)).transpose() # 4x1 - 1x4^Transpose

# G Matrix
G = (sym.Matrix([Vtot]).jacobian(q)).transpose() # 4x1

F_ax = {}
GRF = {}
foot_position = {}

lamda_F_ax = {}
Jfoot = {}
lamda_GRF = {}


foot_position =  sym.Matrix([[r_bases['body'][0]],
                             [r_bases['body'][1]]]).transpose()

F_ax.update({('x'): sym.symbols('F_ax')})
GRF.update({('x'): sym.symbols('GRF_x')})
GRF.update({('y'): sym.symbols('GRF_y')})

Jfoot.update({('foot'): r_bases['body'].jacobian(q)})

lamda_GRF.update({('xy'): sym.Matrix([[GRF['x']],[GRF['y']]])})
lamda_F_ax.update({('x'): sym.Matrix([[F_ax['x']],[0]])})


Q = (Jfoot['foot'].transpose()*lamda_GRF['xy']) + (Jfoot['foot'][0].transpose()*lamda_F_ax['x']) 

In [13]:
Q

⎡Fₐₓ + GRFₓ⎤
⎢          ⎥
⎣  GRF_y   ⎦

In [10]:
# Manipulator Equation 
EOM = M*ddq + G + C - Q 

# save the EOM in a file (saving the EOM saves lots of time)
# import dill
# dill.dump_session('unsimplified_biped_FW.db')

In [11]:
EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    print('{0} is done',j)
    EOMs[j] = EOM[j].simplify()

{0} is done 0
{0} is done 1


In [12]:
# save the EOM in a file (saving the EOM saves lots of time)
import dill
dill.dump_session('simplified_point_mass.db')